In [2]:
import os
import copy
import requests
import numpy as np
import rasterio as rio
from rasterio.plot import show, show_hist
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import ast
%matplotlib inline

In [3]:
def read_tif(tif_file):
    tif_dataset = rio.open(tif_file)
    return tif_dataset
def print_tif(tif_dataset):
    data = tif_dataset.read(1)  # Read the first band
    no_data_value = tif_dataset.nodata
    no_data_count = (data == no_data_value).sum()
    print('dataset:\n',tif_dataset)
    print('\nshape:\n',tif_dataset.shape)
    print('\nno data value:\n',tif_dataset.nodata)
    print('\nno data count:\n',no_data_count)
    print('\nspatial extent:\n',tif_dataset.bounds)
    print('\ncoordinate information (crs):\n',tif_dataset.crs)
def plot_tif(tif_dataset):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))
    show(tif_dataset, ax=ax1);

    show_hist(tif_dataset, bins=50, histtype='stepfilled',
            lw=0.0, stacked=False, alpha=0.3, ax=ax2);
    ax2.set_xlabel(tif_dataset.name.split('/')[-1].split('.')[0]);
    ax2.get_legend().remove()

    plt.show()

def info_tif(tif_dataset):
    print_tif(tif_dataset)
    plot_tif(tif_dataset)

#mode: 0 stands for the larger the value is , the higher risk there will be a landslide, and 1 oppposite
def recalssification_tif(tif_dataset,args,mode=0):
    tif_data = tif_dataset.read(1)
    tif_reclass = tif_data.copy()
    no_class = len(args)+1
    # Get the nodata value from the input raster and set it to 0 in the reclassified raster
    nodata_value = tif_dataset.nodata
    if nodata_value is not None:
        tif_reclass[np.where(tif_data == nodata_value)] = 0
    tif_reclass[np.where(tif_data==-9999.0)] = 0
    if mode == 0:
        if len(args) == 1:
            tif_reclass[np.where((tif_data >= 0) & (tif_data <= args[0]))] = 1 
            tif_reclass[np.where(tif_data > args[0])] = 2
        elif len(args) == 2:
            tif_reclass[np.where((tif_data >= 0) & (tif_data <= args[0]))] = 1 
            tif_reclass[np.where((tif_data > args[0]) & (tif_data <= args[1]))] = 2
            tif_reclass[np.where(tif_data > args[1])] = 3
        elif len(args) == 3:
            tif_reclass[np.where((tif_data >= 0) & (tif_data <= args[0]))] = 1 
            tif_reclass[np.where((tif_data > args[0]) & (tif_data <= args[1]))] = 2
            tif_reclass[np.where((tif_data > args[1]) & (tif_data <= args[2]))] = 3
            tif_reclass[np.where(tif_data > args[2])] = 4
    elif mode == 1:
        if len(args) == 1:
            tif_reclass[np.where((tif_data >= 0) & (tif_data <= args[0]))] = 2 
            tif_reclass[np.where(tif_data > args[0])] = 1
        elif len(args) == 2:
            tif_reclass[np.where((tif_data >= 0) & (tif_data <= args[0]))] = 3 
            tif_reclass[np.where((tif_data > args[0]) & (tif_data <= args[1]))] = 2
            tif_reclass[np.where(tif_data > args[1])] = 1
        elif len(args) == 3:
            tif_reclass[np.where((tif_data >= 0) & (tif_data <= args[0]))] = 4
            tif_reclass[np.where((tif_data > args[0]) & (tif_data <= args[1]))] = 3
            tif_reclass[np.where((tif_data > args[1]) & (tif_data <= args[2]))] = 2
            tif_reclass[np.where(tif_data > args[2])] = 1
    '''
    ext = [tif_dataset.bounds.left,
        tif_dataset.bounds.right,
        tif_dataset.bounds.bottom,
        tif_dataset.bounds.top]
    
    
    # Validate that there are enough indices provided for the classes
    if no_class == 0:
        raise ValueError("At least one class index must be provided for classification.")
    
    # Generate a colormap with `no_class` distinct colors using the 'viridis' colormap
    cmap = plt.get_cmap('viridis', no_class)
    colormap_colors = cmap(np.arange(no_class))

    
    plt.figure(); 
    class_labels = [0,1,2,3]
    
    cmap = colors.ListedColormap(colormap_colors)
    
    # Create custom legend handles
    legend_handles = [
        mpatches.Patch(color=colormap_colors[i], label=class_labels[i])
        for i in range(no_class)
    ]
    #cmap = colors.ListedColormap(['lightblue','yellow','orange','green','red'])
    plt.imshow(tif_reclass,cmap=cmap,extent=ext)
    plt.title(tif_dataset.name.split('/')[-1].split('.')[0])
    ax=plt.gca(); ax.ticklabel_format(useOffset=False, style='plain') 
    rotatexlabels = plt.setp(ax.get_xticklabels(),rotation=90) 
    plt.legend(handles=legend_handles, loc='upper right')
    '''
    return tif_reclass

# Read reclassification information for GLiM 
def read_reclassification_info(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    mapping = ast.literal_eval(lines[0].strip().split('=')[1].strip())
    reclassification_info = {}
    
    for line in lines[2:]:
        if '=' in line:
            key, value = line.strip().split('=')
            reclassification_info[key.strip()] = value.strip()
    
    return mapping, reclassification_info

# Reclassify GLiM file using the reclassification information
def recalssification_glim(data, mapping, reclassification_info, nodata):
    reclassified_data = np.copy(data)
    
    for class_name, new_value in reclassification_info.items():
        if new_value == 'ND':
            new_value = nodata
        else:
            new_value = int(new_value)
        
        short_code = class_name.split('(')[1].split(')')[0].lower()
        if short_code in mapping:
            original_value = mapping[short_code]
            reclassified_data[data == original_value] = new_value
    
    return reclassified_data

#Read reclassification information for landform
def read_reclassification_landform(file_path):
    reclassification_mapping = {}
    
    with open(file_path, 'r') as file:
        for line in file:
            if '=' in line:
                old_value, new_value = line.strip().split('=')
                reclassification_mapping[int(old_value.split(':')[0])] = int(new_value)
    
    return reclassification_mapping

# Reclassify landform file using the reclassification information
def reclassify_raster(data, reclassification_mapping,nodata):
    reclassified_data = np.copy(data)
    
    for old_value, new_value in reclassification_mapping.items():
        if new_value == 'ND':
            new_value = nodata
        else:
            new_value = int(new_value)
        reclassified_data[data == old_value] = new_value
    
    return reclassified_data

def save_reclass_tif(tif_reclass,tif_dataset,save_path=r".\Tsunami_Risk_Zhimin\02_reclassed\on-shore"):
    reclassed_path=save_path
    tif_name=tif_dataset.name.split('/')[-1]
    output_file = os.path.join(reclassed_path, f'reclassified_{tif_name}')
    out_meta = tif_dataset.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": tif_dataset.shape[0],
        "width": tif_dataset.shape[1],
        "transform": tif_dataset.transform,
        "crs": tif_dataset.crs,
        "count": 1,
        "nodata": 0,
        "dtype": 'int32'  
    })
        # Write the reclassified raster to the output file
    with rio.open(output_file, 'w', **out_meta) as dst:
        dst.write(tif_reclass, 1)
    print(f'successfully saved at {output_file}')

In [4]:
# Example usage for reclassification of GLiM data
file_path = r'C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\GLiM.txt'
mapping, reclassification_info = read_reclassification_info(file_path)
print("Mapping:", mapping)
print("Reclassification Info:", reclassification_info)

# change the file path to the location of the GLiM file
glim=r'C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\03_raster_data\coastal\GLiM_on-shore.tif'
glim_data=read_tif(glim)
data=glim_data.read(1)
nodata = glim_data.nodata
reclassified_data = recalssification_glim(data, mapping, reclassification_info, nodata)  
#save reclassified data to given path   
save_path=r'C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\02_reclassed\test'
save_reclass_tif(reclassified_data,glim_data,save_path)

Mapping: {'sm': 1, 'pa': 2, 'mt': 3, 'vi': 4, 'su': 5, 'sc': 6, 'vb': 7, 'ss': 8, 'wb': 9}
Reclassification Info: {'Unconsolidated Sediments (SU)': '3', 'Siliciclastic Sedimentary Rocks (SS)': '2', 'Mixed Sedimentary Rocks (SM)': '2', 'Pyroclastic (PY)': '2', 'Carbonate Sedimentary Rocks (SC)': '2', 'Evaporites (EV)': '2', 'Metamorphic Rocks (MT)': '1', 'Acid Plutonic Rocks (PA)': '1', 'Intermediate Plutonic Rocks (PI)': '1', 'Basic Plutonic Rocks (PB)': '1', 'Acid Volcanic Rocks (VA)': '2', 'Intermediate Volcanic Rocks (VI)': '2', 'Basic Volcanic Rocks (VB)': '2', 'Ice and Glaciers (IG)': '0', 'Water Bodies (WB)': '0', 'No Data (ND)': 'ND'}
successfully saved at C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\02_reclassed\test\reclassified_GLiM_on-shore.tif


In [5]:
# Example usage for reclassification of landform data
file_path = r'C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\landform.txt'
reclassification_mapping = read_reclassification_landform(file_path)
print("Reclassification Mapping:", reclassification_mapping)

# change the file path to the location of the landform file
LF=r'C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\03_raster_data\coastal\Landforms_all.tif'
LF_data=read_tif(LF)
dt=LF_data.read(1)
nodata=-255
rtf=reclassify_raster(dt,reclassification_mapping,nodata)
#save reclassified data to given path   
save_path=r'C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\02_reclassed\test'
save_reclass_tif(rtf,LF_data,save_path)

Reclassification Mapping: {1: 2, 2: 2, 3: 2, 4: 1, 5: 1, 6: 3, 7: 3, 8: 1, 9: 1, 10: 1}
successfully saved at C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\02_reclassed\test\reclassified_Landforms_all.tif


In [1]:
#read threshold information from txt file
with open(r'C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\threshold.txt', 'r') as file:
    lines = file.readlines()

# Now 'lines' is a list where each element is a line from the file
for line in lines:
    print(line.strip())  # .strip() removes the newline character at the end
directories = {}
for line in lines:
    line = line.strip()
    if line:
        line_split=line.split(' ')
        key= line.split(' ')[0]
        value= line.split(' ')[1:]
        directories[key] = value

print(directories)

raster_folder C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\03_raster_data\coastal
save C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\02_reclassed\test
#input:filename,mode(0 stands for the larger the value is , the higher risk there will be a landslide, and 1 oppposite),threshold
chirp_precipitation 0 1800 2200
slope_on-shore 0 27 55
distance_DLR_settlement 1 100 500
#distance_earthquake_all 1 200 1000
distance_faults_all 1 100 500
distance_rivers_all 1 100 500
#distance_valcano_all 1 1000 5000
{'raster_folder': ['C:\\Users\\Zhimin.Chen\\Desktop\\ZChen\\tsunami\\Tsunami_Risk_Zhimin\\03_raster_data\\coastal'], 'save': ['C:\\Users\\Zhimin.Chen\\Desktop\\ZChen\\tsunami\\Tsunami_Risk_Zhimin\\02_reclassed\\test'], '#input:filename,mode(0': ['stands', 'for', 'the', 'larger', 'the', 'value', 'is', ',', 'the', 'higher', 'risk', 'there', 'will', 'be', 'a', 'landslide,', 'and', '1', 'oppposite),threshold'], 'chirp_precipitation': ['0', '1800', '2200'], 'slope_o

In [9]:
# iterates through the directories and reclassifies the tif files
raster_folder = directories['raster_folder'][0]
save_folder = directories['save'][0]

for key in list(directories.keys())[2:]:
    if key[0]=='#':
        print(f'{key[1:]} is a comment, skipping...')
        continue
    tif_key = key
    tif_name=tif_key+'.tif'
    tif_path= os.path.join(raster_folder, tif_name)
    tif_dataset = read_tif(tif_path)   
    tif_data = tif_dataset.read(1)
    no_class = len(directories[key])
    args = directories[key][1:]
    args = [float(i) for i in args]
    tif_mode = int(directories[key][0])
    
    # Convert args to the appropriate data type
    print(tif_dataset,args,tif_mode)
    reclassified_tif = recalssification_tif(tif_dataset,args,mode=tif_mode)
    save_reclass_tif(reclassified_tif, tif_dataset, save_folder)
    print(f'{key} has been reclassified and saved successfully')
    

input:filename,mode(0 is a comment, skipping...
<open DatasetReader name='C:/Users/Zhimin.Chen/Desktop/ZChen/tsunami/Tsunami_Risk_Zhimin/03_raster_data/coastal/chirp_precipitation.tif' mode='r'> [1800.0, 2200.0] 0
successfully saved at C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\02_reclassed\test\reclassified_chirp_precipitation.tif
chirp_precipitation has been reclassified and saved successfully
<open DatasetReader name='C:/Users/Zhimin.Chen/Desktop/ZChen/tsunami/Tsunami_Risk_Zhimin/03_raster_data/coastal/slope_on-shore.tif' mode='r'> [27.0, 55.0] 0
successfully saved at C:\Users\Zhimin.Chen\Desktop\ZChen\tsunami\Tsunami_Risk_Zhimin\02_reclassed\test\reclassified_slope_on-shore.tif
slope_on-shore has been reclassified and saved successfully
<open DatasetReader name='C:/Users/Zhimin.Chen/Desktop/ZChen/tsunami/Tsunami_Risk_Zhimin/03_raster_data/coastal/distance_DLR_settlement.tif' mode='r'> [100.0, 500.0] 1
successfully saved at C:\Users\Zhimin.Chen\Desktop\ZChen\tsun